In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert the images to RGB (3 channels)
x_train = np.stack((x_train,) * 3, axis=-1)
x_test = np.stack((x_test,) * 3, axis=-1)

# Resize the images to 48x48 pixels (the input size of the VGG16 model)
x_train = tf.image.resize(x_train, (48, 48))
x_test = tf.image.resize(x_test, (48, 48))

# Convert the labels to one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
# Load the pre-trained VGG16 model
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Freeze the layers in the VGG16 model
for layer in vgg_model.layers:
    layer.trainable = False

# Add a custom dense layer to the model
x = Flatten()(vgg_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(10, activation='softmax')(x)

# Create a new model with the modified architecture
model = Model(inputs=vgg_model.input, outputs=x)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=32)